# Работа модели для матчинга резюме и вакансий портала "Работа в России":

In [1]:
#Установим библиотеку, разработанную ЦПУР
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8899 --NotebookApp.port_retries=0
!git clone https://github.com/ellariel/rostrud_ml.git

Cloning into 'rostrud_ml'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 75 (delta 30), reused 58 (delta 13), pack-reused 0
Receiving objects: 100% (75/75), 52.18 KiB | 5.22 MiB/s, done.
Resolving deltas: 100% (30/30), done.


После скачивания корневая папка должна быть переименована в rostrud_ml

## Предварительные действия
У этого скрипта есть несколько зависимостей, их можно поставить через pip. При запуске блока кода ниже всё поставится само.

Если в какой-то момент выполнения кода вы увидите ошибку, что вам не хватает некого питоновского пакета, вы можете дописать его название в новую строчку файла requirements.txt и снова запустить этот блок.

In [2]:
#Установка окружения
!pip install -r rostrud_ml/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 66.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


В файле rostrud_ml/utils/all_tables_names.yml указать путь к директории, куда будут скачиваться и парситься файлы с резюме, вакансиями (сейчас там "workdir").

В файле rostrud-ml/utils/config_to_bd_example.yml указать ваши настройки для подключения к БД и поменять название файла на config_to_bd.yml

In [3]:
import requests
from tqdm import tqdm
import pandas as pd

def _getfileslist(url):
  r = requests.get(url)
  return ['data' + i.split('">data')[0] for i in r.text.split('<a href="data')[1:]]

In [4]:
base_url = 'https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/'
file_url = ['7710538364-cv/', '7710538364-professions/', '7710538364-regions/', '7710538364-industries/', '7710538364-vacancy/']

In [6]:
s = pd.DataFrame(_getfileslist(base_url + file_url[0])).rename(columns={0: 'cv'})
s = s[s['cv'].apply(lambda x: pd.notna(x) and '.gz' in x)]
s = s[s['cv'].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
s['date'] = s['cv'].apply(lambda x: x.split('data-')[1][:8])
s = s.groupby('date').agg('last').reset_index()

for l in file_url[1:]:
    c = l.split('-')[1][:2]
    v = pd.DataFrame(pd.Series(_getfileslist(base_url + l)).rename(c))
    v = v[v[c].apply(lambda x: pd.notna(x) and '.gz' in x)]
    v = v[v[c].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
    v['date'] = v[c].apply(lambda x: x.split('data-')[1][:8])
    v = v.groupby('date').agg('last').reset_index()
    s = s.set_index('date').join(v.set_index('date')).reset_index()
s.to_csv('filtered_all.csv', index=False)

## Загрузка в БД необходимых данных

In [8]:
filtered = pd.read_csv('filtered_all.csv') #список ссылок полученный на прошлом этапе
len(filtered)

1308

In [ ]:
#Установка модулей
from rostrud_ml.process.adding_tables_psycopg import AddingDataPsycopg
from rostrud_ml.process.renewal import Renewal

In [ ]:
#Cоздание таблиц в БД
db = AddingDataPsycopg()
for table in ['curricula_vitae', 'workexp', 'vacancies', 'professions', 'industries', 'regions']:
    db.create_table(table, 'project_trudvsem') #'project_trudvsem' - название схемы в вашей БД, создать, если нету
db.conn.close()

Взять ссылки на файлы с данными с ftp поратала https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/:

Из директорий
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-cv/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-vacancy/ нужно выбрать подходящие файлы (имеющие в наименовании дату) - один файл резюме и один вакансий.

Подобным образом выбрать ссылки для данных о профессиях, сферах занятости, регионах из соответствующих директорий:
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-professions/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-industries/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-regions/

Для каждого типа, вставив нужную ссылку, запустить следующие ячейки:
В примере: тип "резюме" и самый первый файл резюме 2018г. Необходимо поменять ссылку на наиболее актуальный файл.

In [10]:
filtered.head(1)

,date,cv,pr,re,in,va
0,20190101,data-20190101T031204-structure-20161130T143000...,data-20190101T031204-structure-20161130T143000...,data-20190101T031204-structure-20161130T143000...,data-20190101T031204-structure-20161130T143000...,data-20190101T031204-structure-20161130T143000...


In [ ]:
for _, link in tqdm(filtered.iterrows(), total=len(filtered)):
  #if int(link['month']) < 202005:
  #  continue
  try:
    ren = Renewal('cvs', base_url + file_url[0] + link['cv'])
    ren.download()
    ren.extract()
    ren.parse_update()
    ren.delete()
  except Exception as e:
    print(f'cvs error: {e}')

  if pd.notna(link['va']):
    try:
      ren = Renewal('vacancies', base_url + file_url[4] + link['va'])
      ren.download()
      ren.extract()
      ren.parse_update()
      ren.delete()
    except Exception as e:
      print(f'vacancies error: {e}')

  try:
    ren = Renewal('professions', base_url + file_url[1] + link['pr'])
    ren.download()
    ren.extract()
    ren.parse_update()
    ren.delete()
  except Exception as e:
    print(f'professions error: {e}')

  try:
    ren = Renewal('industries', base_url + file_url[2] + link['re'])
    ren.download()
    ren.extract()
    ren.parse_update()
    ren.delete()
  except Exception as e:
    print('industries error')

  try:
    ren = Renewal('regions', base_url + file_url[3] + link['in'])
    ren.download()
    ren.extract()
    ren.parse_update()
    ren.delete()
  except Exception as e:
    print('regions error')

  #break